In [1]:
import cv2
import argparse
import numpy as np
import utils

In [2]:
# read input image
image_file = "object-detection-opencv/test.jpg"
image_file2 = "object-detection-opencv/test_no_person.jpg"
# list_images = utils.get_files_by_ext("object-detection-opencv/", "jpg")
list_images = [image_file, image_file2]
classes_file = "object-detection-opencv/yolov3.txt"
weights_file = "object-detection-opencv/yolov3.weights"
conf_file = "object-detection-opencv/yolov3.cfg"

Width = list()
Height = list()
for image_path in list_images:
    image = cv2.imread(image_path)

    Width.append(image.shape[1])
    Height.append(image.shape[0])

scale = 0.00392

# read class names from text file
classes = None
with open(classes_file, 'r') as f:
    classes = [line.strip() for line in f.readlines()]

# generate different colors for different classes 
COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

blob = utils.image_to_matrix(list_images)

# read pre-trained model and config file
# reads the weights and config file and creates the network.
net = cv2.dnn.readNet(weights_file, conf_file)

# create input blob 
# prepares the input image to run through the deep neural network
    
# set input blob for the network
net.setInput(blob)

In [3]:
# function to get the output layer names 
# in the architecture
def get_output_layers(net):
    
    layer_names = net.getLayerNames()
    
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    return output_layers

# function to draw bounding box on the detected object with class name
def draw_bounding_box(img, class_id, confidence, x, y, x_plus_w, y_plus_h):

    label = str(classes[class_id])

    color = COLORS[class_id]

    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)

    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

In [4]:
def init_detect_person():
    # initialization
    conf_threshold = 0.5
    nms_threshold = 0.4
    
    return conf_threshold, nms_threshold

In [52]:
def detect_object(outs, list_images, Width, Height):

    i = 0

    # for each detetion from each output layer 
    # get the confidence, class id, bounding box params
    # and ignore weak detections (confidence < 0.5)
    for out in outs:
        for image in out:
            image_name = list_images[i]
            if not image_name in dict_obj_detected:
                dict_image_danger[image_name] = {}
                dict_obj_detected[image_name] = {}
                dict_obj_detected[image_name]["class_ids"] = list()
                dict_obj_detected[image_name]["confidences"] = list()
                dict_obj_detected[image_name]["boxes"] = list()
            for detection in image:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:
                    center_x = int(detection[0] * Width[i])
                    center_y = int(detection[1] * Height[i])
                    w = int(detection[2] * Width[i])
                    h = int(detection[3] * Height[i])
                    x = center_x - w / 2
                    y = center_y - h / 2
                    dict_obj_detected[image_name]["class_ids"].append(class_id)
                    dict_obj_detected[image_name]["confidences"].append(float(confidence))
                    dict_obj_detected[image_name]["boxes"].append([x, y, w, h])
            i += 1
        i = 0 

    return dict_obj_detected

def detect_danger(dict_obj_detected, idx_class=0):
    
    for image_name, row in dict_obj_detected.items():
        if len(row["class_ids"]) != 0 and idx_class in row["class_ids"]: 
            dict_image_danger[image_name] = True
        else:
            dict_image_danger[image_name] = False
    
    return dict_image_danger

In [54]:
class_ids, confidences, boxes, conf_threshold, nms_threshold = init_detect_person()

# run inference through the network
# and gather predictions from output layers
outs = net.forward(get_output_layers(net))
dict_obj_detected = {}
dict_obj_detected = detect_object(outs, list_images, Width, Height)
dict_danger = detect_danger(dict_obj_detected)
print(dict_danger)

NameError: name 'dict_image_danger' is not defined

In [ ]:
# apply non-max suppression
indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
print(indices)
# go through the detections remaining
# after nms and draw bounding box
for i in indices:
    i = i[0]
    box = boxes[i]
    x = box[0]
    y = box[1]
    w = box[2]
#     h = box[3]
    
    draw_bounding_box(image, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))

# # display output image
# cv2.imshow("object detection", image)

# # wait until any key is pressed
# cv2.waitKey()
    
#  # save output image to disk
cv2.imwrite("object-detection.jpg", image)

# # release resources
# cv2.destroyAllWindows()

In [ ]:
# get all of the results above a threshold
def get_boxes(boxes, labels, thresh):
    v_boxes, v_labels, v_scores = list(), list(), list()
    # enumerate all boxes
    for box in boxes:
        # enumerate all possible labels
        for i in range(len(labels)):
            # check if the threshold for this label is high enough
            if box.classes[i] > thresh:
                v_boxes.append(box)
                v_labels.append(labels[i])
                v_scores.append(box.classes[i]*100)
                # don't break, many labels may trigger for one box
    return v_boxes, v_labels, v_scores

In [ ]:
for confidence in confidences:
    print(confidence)